In [ ]:
!python --version

In [ ]:
!pwd

In [ ]:
CONFIG = 'basic_rnn'
BUNDLE_PATH = '/root/magenta/magenta/models/melody_rnn/mag/basic_rnn.mag'
OUTPUT_DIR = '/root/magenta/magenta/models/melody_rnn/output/2020.3.7-01'

# 用 pre-trained model 生成序列

In [ ]:
!melody_rnn_generate \
--config='attention_rnn' \
--bundle_file='/root/magenta/magenta/models/melody_rnn/mag/attention_rnn.mag' \
--output_dir='/root/magenta/magenta/models/melody_rnn/output/2020.3.30' \
--num_outputs=5 \
--num_steps=256 \
--primer_midi='/root/magenta/magenta/models/melody_rnn/primer.mid'

# 制作数据集

**注：这一节的脚本只是记录下来的，由于输出过多，VS Code容易与服务器断开连接，所以脚本需要在终端运行。**

## Building your Dataset

In [ ]:
INPUT_DIRECTORY=/download/clean_midi

# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD=/root/notesequences.tfrecord

convert_dir_to_note_sequences \
  --input_dir=$INPUT_DIRECTORY \
  --output_file=$SEQUENCES_TFRECORD \
  --recursive

## Create SequenceExamples

In [ ]:
melody_rnn_create_dataset \
--config=attention_rnn \
--input=/root/notesequences.tfrecord \
--output_dir=/download/melody_rnn/sequence_examples \
--eval_ratio=0.10

# 训练模型

## 第一次训练(3.30-3.31): run1

In [ ]:
melody_rnn_train \
--config=attention_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run1 \
--sequence_example_file=/download/melody_rnn/sequence_examples/training_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--num_training_steps=20000

run1因为未知原因被中止。生成旋律：

In [ ]:
!melody_rnn_generate \
--config=attention_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run1 \
--output_dir=/root/magenta/magenta/models/melody_rnn/generated/run1 \
--num_outputs=10 \
--num_steps=128 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--primer_melody="[60]"

## 生成bundle file(.mag文件)

In [ ]:
!melody_rnn_generate \
--config=attention_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run1 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--bundle_file=/root/magenta/magenta/models/melody_rnn/generated/run1/attention_rnn.mag \
--save_generator_bundle

## 第二次训练(4.4): run2

In [ ]:
melody_rnn_create_dataset \
--config=basic_rnn \
--input=/root/notesequences.tfrecord \
--output_dir=/download/basic_rnn/sequence_examples \
--eval_ratio=0.10

**制作数据集出错：**

> Traceback (most recent call last):
  File "/root/anaconda3/envs/magenta/bin/melody_rnn_create_dataset", line 11, in <module>
    load_entry_point('magenta', 'console_scripts', 'melody_rnn_create_dataset')()  File "/root/magenta/magenta/models/melody_rnn/melody_rnn_create_dataset.py", line 63, in console_entry_point
    tf.app.run(main)
  File "/root/anaconda3/envs/magenta/lib/python3.7/site-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/root/anaconda3/envs/magenta/lib/python3.7/site-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/root/anaconda3/envs/magenta/lib/python3.7/site-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/root/magenta/magenta/models/melody_rnn/melody_rnn_create_dataset.py", line 59, in main
    FLAGS.output_dir)
  File "/root/magenta/magenta/pipelines/pipeline.py", line 374, in run_pipeline_serial
    for input_ in input_iterator:
  File "/root/magenta/magenta/pipelines/pipeline.py", line 310, in tf_record_iterator
    for raw_bytes in tf.python_io.tf_record_iterator(tfrecord_file):
  File "/root/anaconda3/envs/magenta/lib/python3.7/site-packages/tensorflow_core/python/lib/io/tf_record.py", line 181, in tf_record_iterator
    reader.GetNext()
  File "/root/anaconda3/envs/magenta/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow_internal.py", line 1034, in GetNext    return _pywrap_tensorflow_internal.PyRecordReader_GetNext(self)tensorflow.python.framework.errors_impl.DataLossError: truncated record at 1412079946

In [ ]:
melody_rnn_train \
--config=basic_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run2 \
--sequence_example_file=/download/basic_rnn/sequence_examples/training_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--num_training_steps=20000

训练成功，测试生成旋律。

**eval**

In [ ]:
melody_rnn_train \
--config=basic_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run2 \
--sequence_example_file=/download/basic_rnn/sequence_examples/eval_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--num_training_steps=20000 \
--eval

In [1]:
!melody_rnn_generate \
--config=basic_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run2 \
--output_dir=/root/magenta/magenta/models/melody_rnn/generated/run2 \
--num_outputs=10 \
--num_steps=128 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--primer_melody="[60]"

INFO:tensorflow:hparams = {'batch_size': 64, 'rnn_layer_sizes': [64, 64], 'dropout_keep_prob': 0.5, 'attn_length': 0, 'clip_norm': 5, 'learning_rate': 0.001, 'residual_connections': False, 'use_cudnn': False}
I0404 18:08:54.359242 140701875304256 events_rnn_graph.py:226] hparams = {'batch_size': 64, 'rnn_layer_sizes': [64, 64], 'dropout_keep_prob': 0.5, 'attn_length': 0, 'clip_norm': 5, 'learning_rate': 0.001, 'residual_connections': False, 'use_cudnn': False}
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0404 18:08:54.362559 140701875304256 deprecation.py:323] From /root/magenta/magenta/models/shared/events_rnn_graph.py:57: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This c